# Day 09. Exercise 01
# Gridsearch

## 0. Imports

In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from tqdm.notebook import tqdm
import numpy as np
from sklearn.metrics import accuracy_score

## 1. Preprocessing

1. Read the file [`day-of-week-not-scaled.csv`](https://drive.google.com/file/d/1AlGvsJDSzPT_70caausx8bFuupIEZkfh/view?usp=sharing). It is similar to the one from the previous exercise, but this time we did not scale continuous features (we are not going to use logreg anymore).
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [3]:
df = pd.read_csv('../data/day-of-week-not-scaled.csv')
df

,numTrials,hour,dayofweek,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,...,labname_lab02,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1
0,1,5,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,5,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,3,5,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,5,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,5,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,9,20,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1682,6,20,3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1683,7,20,3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1684,8,20,3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [7]:
X = df.drop('dayofweek', axis=1)
y = df['dayofweek']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

## 2. SVM gridsearch

1. Using `GridSearchCV` try different parameters of kernel (`linear`, `rbf`, `sigmoid`), C (`0.01`, `0.1`, `1`, `1.5`, `5`, `10`), gamma (`scale`, `auto`), class_weight (`balanced`, `None`) use `random_state=21` and `probability=True` and get the best combination of them in terms of accuracy.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`. Check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [10]:
svc = SVC(random_state=21, probability=True)

In [11]:
param_grid = {'C': [0.01, 0.1, 1, 1.5, 5, 10],
              'kernel': ['linear', 'rbf', 'sigmoid'],
              'gamma': ['scale', 'auto'],
              'class_weight': ['balanced', None]
              }

gs = GridSearchCV(svc, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train, y_train)
print(f'лучшие параметры: {gs.best_params_}')
print(f'лучший score: {gs.best_score_}')

лучшие параметры: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf'}
лучший score: 0.8761090458488228


In [13]:
results = pd.DataFrame(gs.cv_results_)
results = results.sort_values('rank_test_score', ascending=True)
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
70,1.229199,0.007652,0.070200,0.001939,10,None,auto,rbf,"{'C': 10, 'class_weight': None, 'gamma': 'auto...",0.900000,0.848148,0.885185,0.884758,0.862454,0.876109,0.018419,1
64,1.268198,0.028265,0.073600,0.003878,10,balanced,auto,rbf,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.877778,0.851852,0.862963,0.873606,0.851301,0.863500,0.010870,2
58,1.140598,0.015213,0.074000,0.000632,5,None,auto,rbf,"{'C': 5, 'class_weight': None, 'gamma': 'auto'...",0.825926,0.811111,0.818519,0.821561,0.802974,0.816018,0.008116,3
52,1.201599,0.019765,0.076600,0.002577,5,balanced,auto,rbf,"{'C': 5, 'class_weight': 'balanced', 'gamma': ...",0.844444,0.785185,0.792593,0.817844,0.802974,0.808608,0.021007,4
63,98.189905,9.204859,0.021201,0.000399,10,balanced,auto,linear,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.729630,0.700000,0.755556,0.754647,0.665428,0.721052,0.034438,5


## 3. Decision tree

1. Using `GridSearchCV` try different parameters of `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use `random_state=21`.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [16]:
tree = DecisionTreeClassifier(random_state=21)

In [18]:
param_grid = {'criterion': ['gini','entropy'],
              'max_depth': np.arange(1, 50),
              'class_weight': ['balanced', None]
            }

gs = GridSearchCV(tree, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train, y_train)
print(f'лучшие параметры: {gs.best_params_}')
print(f'лучший score: {gs.best_score_}')

лучшие параметры: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 22}
лучший score: 0.8731212997384002


In [19]:
results = pd.DataFrame(gs.cv_results_)
results = results.sort_values('rank_test_score', ascending=True)
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
21,0.016400,0.000491,0.004400,4.916355e-04,balanced,gini,22,"{'class_weight': 'balanced', 'criterion': 'gin...",0.885185,0.862963,0.903704,0.881041,0.832714,0.873121,0.023998,1
20,0.015996,0.000632,0.004001,7.747687e-07,balanced,gini,21,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.859259,0.903704,0.884758,0.828996,0.873121,0.026300,2
29,0.016400,0.000800,0.004200,3.998760e-04,balanced,gini,30,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.832714,0.873116,0.023911,3
32,0.016001,0.000894,0.004200,3.987317e-04,balanced,gini,33,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.832714,0.873116,0.023911,3
28,0.017600,0.002245,0.004001,1.177701e-06,balanced,gini,29,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.832714,0.873116,0.023911,3


## 4. Random forest

1. Using `GridSearchCV` try different parameters of `n_estimators` (`5`, `10`, `50`, `100`), `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use random_state=21.
2. Create a dataframe from the results of the gridsearch and sort it ascendengly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [20]:
frt = RandomForestClassifier(random_state=21)

In [21]:
param_grid = {'n_estimators': [5, 10, 50, 100],
              'criterion': ['gini','entropy'],
              'max_depth': np.arange(1, 50),
              'class_weight': ['balanced', None],
              'random_state': [21]}

gs = GridSearchCV(frt, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train, y_train)
print(f'лучшие параметры: {gs.best_params_}')
print(f'лучший score: {gs.best_score_}')

лучшие параметры: {'class_weight': None, 'criterion': 'gini', 'max_depth': 28, 'n_estimators': 50, 'random_state': 21}
лучший score: 0.9042902381935839


In [22]:
results = pd.DataFrame(gs.cv_results_)
results = results.sort_values('rank_test_score', ascending=True)
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,param_n_estimators,param_random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
502,0.287800,0.002786,0.020200,0.000400,None,gini,28,50,21,"{'class_weight': None, 'criterion': 'gini', 'm...",0.922222,0.900000,0.907407,0.903346,0.888476,0.904290,0.010961,1
515,0.574600,0.005499,0.040200,0.003487,None,gini,31,100,21,"{'class_weight': None, 'criterion': 'gini', 'm...",0.918519,0.911111,0.900000,0.910781,0.877323,0.903547,0.014380,2
118,0.306201,0.003655,0.021199,0.001939,balanced,gini,30,50,21,"{'class_weight': 'balanced', 'criterion': 'gin...",0.922222,0.907407,0.881481,0.907063,0.895911,0.902817,0.013554,3
134,0.305999,0.006388,0.020600,0.001357,balanced,gini,34,50,21,"{'class_weight': 'balanced', 'criterion': 'gin...",0.922222,0.907407,0.892593,0.907063,0.884758,0.902809,0.013010,4
587,0.575800,0.010323,0.036400,0.000490,None,gini,49,100,21,"{'class_weight': None, 'criterion': 'gini', 'm...",0.914815,0.911111,0.900000,0.903346,0.884758,0.902806,0.010460,5


## 5. Progress bar

Gridsearch can be a quite long process and you may find yourself wondering when it will end.
1. Create a manual gridsearch for the same parameters values of random forest iterating through the list of the possible values and calculating `cross_val_score` for each combination. Try to increase `n_jobs`. The value `cv` for `cross_val_score` is 5.
2. Track the progress using the library `tqdm.notebook`.
3. Create a dataframe from the results of the gridsearch with the columns corresponding to the names of the parameters and `mean_accuracy` and `std_accuracy`.
4. Sort it descendingly by the `mean_accuracy`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [36]:
grid = list(ParameterGrid(param_grid))
print(f'Количество комбинаций суперпараметров: {len(grid)}')

Количество комбинаций суперпараметров: 784


In [39]:
data = []

for params in tqdm(grid):
    d = {}
    estimator = RandomForestClassifier(**params)
    sc = cross_val_score(estimator, X_train, y_train, cv=5, n_jobs=1)
    d = {**params, 'mean_accuracy': np.mean(sc), 'std_accuracy': np.std(sc)}
    data.append(d)

  0%|          | 0/784 [00:00<?, ?it/s]

In [40]:
result = pd.DataFrame(data)
result = result.sort_values('mean_accuracy', ascending=False)
result

,class_weight,criterion,max_depth,n_estimators,random_state,mean_accuracy,std_accuracy
502,None,gini,28,50,21,0.904290,0.010961
515,None,gini,31,100,21,0.903547,0.014380
118,balanced,gini,30,50,21,0.902817,0.013554
134,balanced,gini,34,50,21,0.902809,0.013010
551,None,gini,40,100,21,0.902806,0.010460
...,...,...,...,...,...,...,...
588,None,entropy,1,5,21,0.353832,0.016467
200,balanced,entropy,2,5,21,0.353110,0.021165
4,balanced,gini,2,5,21,0.346419,0.029749
0,balanced,gini,1,5,21,0.283390,0.011062


## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.

In [41]:
rfc = RandomForestClassifier(n_estimators=50, max_depth=28, criterion='gini', class_weight=None, random_state=21)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [44]:
accuracy_score(y_test, y_pred)

0.9289940828402367